### PREPARE THE STATS RUN

In [ ]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

season_map_df = pd.DataFrame({
    'Season': [3, 4, 6, 9, 12, 15, 18, 20, 22, 24, 26, 27, 28],
    'Season Year': [
        '2010-11', '2011-12', '2012-13', '2013-14', '2014-15',
        '2015-16', '2016-17', '2017-18', '2018-19', '2019-20',
        '2020-21', '2021-22', '2022-23'
    ]
})


# Base URL
base_url = "https://stats.caha.timetoscore.com/display-stats?league=3"

# Initialize a list to store the data
data = []

# Iterate through the season map
for index, row in season_map_df.iterrows():
    season_year = row['Season Year']
    season_number = row['Season']
    season_url = f"{base_url}&season={season_number}"
    
    # Fetch the content of the season URL
    response = requests.get(season_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all the links on the page
        links = soup.find_all('a')
        
        # Initialize lists to store schedule names and division player stats URLs
        schedule_names = []
        division_player_stats_urls = []
        
        # Extract Schedule and Division Player Stats URLs
        for link in links:
            if 'Division Player Stats' in link.get_text():
                division_player_stats_url = urljoin(base_url, link['href'])
                division_player_stats_urls.append(division_player_stats_url)
            elif 'Schedule' in link.get_text() and 'Norcal Schedule' not in link.get_text():
                schedule_name = link.get_text()
                schedule_names.append(schedule_name)
        
        # Append data to the list
        data.extend([
            {
                'Season Name': season_year,
                'Division': schedule_name.replace(' Schedule', '').strip(),
                'Division Player Stats URL': division_player_stats_url
            }
            for schedule_name, division_player_stats_url in zip(schedule_names, division_player_stats_urls)
        ])

# Create a DataFrame from the collected data
division_list_df = pd.DataFrame(data)


### RETRIEVE PLAYER STATS FROM ALL SEASONS

In [ ]:
def scrape_division_stats(url, season_name, division):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table')
        
        if table:
            division_stats_df = pd.read_html(str(table), header=0)[0]
            division_stats_df.insert(0, 'Season Name', season_name)
            division_stats_df.insert(1, 'Division', division)
            return division_stats_df
    
    return None

# Initialize a list to store the data
data = []

# Iterate through the division list
for index, row in division_list_df.iterrows():
    season_name = row['Season Name']
    division = row['Division']
    division_player_stats_url = row['Division Player Stats URL']
    
    division_stats_df = scrape_division_stats(division_player_stats_url, season_name, division)
    
    if division_stats_df is not None:
        data.append(division_stats_df)

### Clean Player Stats

In [ ]:
# Concatenate all DataFrames in the list if there's any data
if data:
    result_df = pd.concat(data, ignore_index=True)
    
    # Remove duplicate lines
    result_df.drop_duplicates(inplace=True)
 
    # Define the new column names
    new_column_names = ['Season Name', 'Division', 'Name', '#', 'Team', 'GP', 'Goals', 'Ass.', 'Hat', 'Min', 'Pts/Game', 'Pts']

    # Update the column names with the new defined names
    result_df.columns = new_column_names

    # Remove any redundant rows where the "Name" column has the value "Name", starting from the second row
    result_df = result_df.loc[(result_df['Name'] != 'Name') | (result_df.index == 1)]

    # Write the cleaned DataFrame to a CSV file
    result_df.to_csv('norcal_player_stats.csv', index=False)

else:
    print("No valid data found.")


### RETRIEVE GOALIE STATS FROM ALL SEASONS


In [ ]:
def scrape_goalie_stats(url, season_name, division):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tables = soup.find_all('table')

        if len(tables) > 1:
            goalie_stats_df = pd.read_html(str(tables[1]), header=0)[0]
            goalie_stats_df.insert(0, 'Season Name', season_name)
            goalie_stats_df.insert(1, 'Division', division)
            return goalie_stats_df
    return None

# Initialize a list to store the scraped data
data = []

# Iterate through the division list and scrape goalie stats
for index, row in division_list_df.iterrows():
    season_name = row['Season Name']
    division = row['Division']
    division_player_stats_url = row['Division Player Stats URL']
    goalie_stats_df = scrape_goalie_stats(division_player_stats_url, season_name, division)
    if goalie_stats_df is not None:
        data.append(goalie_stats_df)

### Clean Goalie Stats

In [ ]:
# Concatenate and clean the scraped data
if data:
    goalie_stats_df = pd.concat(data, ignore_index=True)
    goalie_stats_df.drop_duplicates(inplace=True)

    # Define the new column names
    new_column_names = ['Season Name', 'Division', 'Name', 'Team', 'GP', 'Shots', 'GA', 'GAA', 'Save %', 'SO']

    # Update the column names with the new defined names
    goalie_stats_df.columns = new_column_names

    # Remove any redundant rows where the "Name" column has the value "Name", starting from the second row
    goalie_stats_df = goalie_stats_df.loc[(goalie_stats_df['Name'] != 'Name') | (goalie_stats_df.index == 1)]

    # Write the cleaned DataFrame to a CSV file
    goalie_stats_df.to_csv('norcal_goalie_stats.csv', index=False)

else:
    print("No valid data found. Skipping writing to norcal_goalie_stats.csv")